In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [2]:
import sys
sys.path.insert(1, "../../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.yolov8cls_path import Model
from evaluation import count_parameters

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [6]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


# Inference at 640x640

In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((640, 640)),
                                      transforms.Normalize(mean=means, std=stds)])

In [8]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [9]:
img = train_dataset[0][0].unsqueeze(0)

In [10]:
from torch.profiler import profile, ProfilerActivity

In [11]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [12]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_640'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
    for i in range(12):
        out = model(img)
        prof.step()
print(prof.key_averages().table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   ProfilerStep*         0.76%       9.953ms       100.00%        1.315s     131.480ms     210.21 Mb      -1.90 Gb            10            --  
                      Conv block         5.69%      74.864ms        93.40%        1.228s       4.723ms       1.93 Gb           0 b           260            --  
                    aten::conv2d         0.17%       2.250ms        61.75%     811.913ms       3.123ms     660.16 Mb           0 b           260    124452.864  
               aten::convolution  

# Inference at 640x640 using Ultralytics functions

In [13]:
from ultralytics_flops import get_flops, get_flops_with_torch_profiler

In [14]:
get_flops(model, imgsz=640)

13.609779199999998

In [15]:
get_flops_with_torch_profiler(model, imgsz=640)

12.7034368

# Inference at 224x224

In [23]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [24]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

In [25]:
img = train_dataset[0][0].unsqueeze(0)

In [26]:
from torch.profiler import profile, ProfilerActivity

In [27]:
model = Model(num_classes=1000, 
              residual_connection=True, 
              CSP=True, 
              add_hidden=True,
              classifyV8=True,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [44]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-5/inference_at_224'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
    for i in range(12):
        out = model(img)
        prof.step()
print(prof.key_averages().table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total MFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   ProfilerStep*         0.75%      12.339ms       100.00%        1.643s     164.315ms      13.30 Mb      -2.10 Gb            10            --  
                      Conv block         6.49%     106.631ms        93.46%        1.536s       5.907ms       1.93 Gb           0 b           260            --  
                    aten::conv2d         0.18%       3.025ms        60.24%     989.766ms       3.807ms     660.16 Mb           0 b           260    124452.864  
               aten::convolution  

# Count Parameters

My implementation:

In [30]:
count_parameters(model)

6361736

Ultralytics implementation:

In [36]:
model = torch.load('../../../Models/yolov8s-cls.pt')['model'].model

In [37]:
count_parameters(model)

6361736